In [53]:
import pandas as pd
import os
import glob
import joblib
from notion_client import Client
from notion_client.helpers import is_full_page
import config
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS, ASYNCHRONOUS
import argparse
import datetime
from datetime import tzinfo



config_file = 'config.ini'

path = 'transactions/sept28_2024/'


cfg = config.Config(config_file)

In [11]:
all_files = glob.glob(os.path.join(path, "*.csv"))

df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

df['Date'] = df['Transaction date'].combine_first(df['Date'])
df.pop('Transaction date')
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')


df['hash'] = df.apply(joblib.hash, axis=1)
df

,Transaction,Name,Memo,Amount,Date,hash
0,DEBIT,SQ *SEALY KARATE SCHOO Guelph ON,Rewards earned: 0.79 ~ Category: Other,-158.20,2024-09-26,b0474e5415ee1321e19a8874b5edb85d
1,DEBIT,TERRY FOX* COLE LMENTA BURNABY B,Rewards earned: 0.13 ~ Category: Other,-25.00,2024-09-26,e35c38584d66e2327cc0d6ea84a30166
2,DEBIT,ZEHRS #38 - ERAMOSA GUELPH ON,Rewards earned: 0.51 ~ Category: Groceries,-25.59,2024-09-25,1175cfadac37048cb86a9d545d93e6e2
3,DEBIT,MCDONALD'S #18461 WATERLOO ON,Rewards earned: 0.16 ~ Category: Restaurant,-7.90,2024-09-25,81a0e02e182dcb076643c45af21cb03d
4,DEBIT,FRESHCO #3847 GUELPH ON,Rewards earned: 4.01 ~ Category: Groceries,-200.47,2024-09-23,116f0bf12aeb6ae207ce89f7f5ca78b0
5,DEBIT,PUR SIMPLE GUELPH GUELPH ON,Rewards earned: 0.57 ~ Category: Restaurant,-28.27,2024-09-23,4bb12ed61772446d60ac36539e98531d
6,DEBIT,SQ *HC APPLEZ Guelph ON,Rewards earned: 0.07 ~ Category: Other,-14.00,2024-09-22,db61391d399751612b10f9eb3bd708f7
7,DEBIT,SQ *HC APPLEZ Guelph ON,Rewards earned: 0.09 ~ Category: Other,-18.00,2024-09-22,9657a0d4bbb52346f1171b61c2cdf959
8,DEBIT,7-ELEVEN #29092 (BULLO GUELPH ON,Rewards earned: 0.21 ~ Category: Gas,-41.44,2024-09-21,a44978112cbc90e079e9e64664b2126e
9,DEBIT,BLUEBIRD CAFE & GRILL ORANGEVILL,Rewards earned: 0.97 ~ Category: Restaurant,-48.44,2024-09-21,81e65a64bbbefc6e65f60b00b6999aca


In [30]:

notion_secret = cfg.config['TANGERINE']['notion_secret']
notion_database = cfg.config['TANGERINE']['notion_database']
notion = Client(auth=notion_secret)

full_or_partial_pages = notion.databases.query(database_id=notion_database)
rules = []
for page in full_or_partial_pages["results"]:
    if not is_full_page(page):
        continue
    try:
        #print(page)
        #print(f"Created at: {page['created_time']}")
        if len(page['properties']['Replacement']['multi_select']):
            replacement = page['properties']['Replacement']['multi_select'][0]['name']
        else:
            replacement = None
        method = 'exact'
        key = None
        for t in page['properties']['Type']['multi_select']:
            if t['name'] == 'in':
                method = 'in'
            else:
                key = t['name']
        title = page['properties']['Rule']['title'][0]['text']['content']
        d = dict(replacement=replacement, title=title, method=method, key=key)
        rules.append(d)
    except Exception as e:
        print(e)
        continue

print('notion rules', rules)
filter_rules = list(filter(lambda d: d['replacement'] is None, rules))

notion rules [{'replacement': 'Water Heater', 'title': 'Reliance', 'method': 'in', 'key': 'merchant_name'}, {'replacement': 'Shops', 'title': 'Discount Stores', 'method': 'exact', 'key': 'category'}, {'replacement': 'Kids Activities', 'title': 'Holly Hughes Dance', 'method': 'in', 'key': 'name'}, {'replacement': None, 'title': 'Internet Deposit from Tangerine Chequing Account', 'method': 'in', 'key': 'name'}, {'replacement': None, 'title': 'Internet Withdrawal to Tangerine Savings Account', 'method': 'in', 'key': 'name'}, {'replacement': 'Hobbies', 'title': 'Georgia Simms', 'method': 'in', 'key': 'name'}, {'replacement': 'Mortgage', 'title': 'RBC PYT', 'method': 'in', 'key': 'name'}, {'replacement': 'Groceries', 'title': 'Goodfood', 'method': 'exact', 'key': 'merchant_name'}, {'replacement': None, 'title': 'PAYMENT - THANK YOU', 'method': 'exact', 'key': 'name'}, {'replacement': None, 'title': 'Payment', 'method': 'exact', 'key': 'merchant_name'}, {'replacement': 'Restaurants', 'title'

In [55]:
class Transaction:
    def __init__(self, data, rules=[]):
        self.raw_data = data
        if not data['merchant_name']:
            data['merchant_name'] = data['name']
        old = data['category'][-1]
        for rule in rules:
            if rule['replacement'] is None:
                continue
            if rule['key'] in ['merchant_name', 'name']:
                if rule['method'] == 'exact' and data[rule['key']] == rule['title']:
                    data['category'][-1] = rule['replacement']
                elif rule['method'] == 'in' and rule['title'] in data[rule['key']]:
                    data['category'][-1] = rule['replacement']
            elif rule['key'] == 'category':
                if rule['method'] == 'exact' and data[rule['key']][-1] == rule['title']:
                    data['category'][-1] = rule['replacement']
                elif rule['method'] == 'in' and rule['title'] in data[rule['key']][-1]:
                    data['category'][-1] = rule['replacement']     
                    
        new = data['category'][-1]
        #if old != new:
        #    print(data['name'], old,  new)
        self.account_id     = data['account_id']
        self.date           = data['authorized_date'] if data['authorized_date'] is not None else data['date']
        self.name           = data['name']        
        #print(self.name, data['date'], data['amount'] , data['transaction_id'])
        self.transaction_id = data['transaction_id']
        self.pending        = data['pending']
        self.merchant_name  = data['merchant_name']
        self.amount         = data['amount']
        self.currency_code  = data['iso_currency_code']
        self.category       = data['category']

    def __str__(self):
        return "%s %s %s (%s) - %4.2f %s" % ( self.date, self.transaction_id, self.merchant_name, self.name, self.amount, self.category[-1])
    def __repr__(self):
        return self.__str__()


transactions = []
for _, row in df.iterrows():
    category = row['Memo'].split('Category:')
    if len(category) == 2:
        category = category[-1]
    else:
        category = 'Other'
    t = dict(name=row['Name'], category=[category], date=row['Date'], account_id='NA', transaction_id=row['hash'], pending=False, iso_currency_code='CA', amount=row['Amount'], merchant_name=None, authorized_date=None)
    for rule in filter_rules:
        skip = False
        if rule['method'] == 'exact' and rule['title'] == t[rule['key']]:
            skip = True
            break
        elif rule['method'] == 'in' and rule['title'] in t[rule['key']]:
            skip = True
            break
        if t['pending']:
            skip = True
        if skip:
            continue
    transactions.append(Transaction(t, rules))

In [58]:
def upload(transactions, cfg, start_date, end_date, verbose=False, dump=False, dry=False):
    old = {}
    client = InfluxDBClient(url=cfg.config['TANGERINE']['influx_url'], token=cfg.config['TANGERINE']['influx_token'],  org=cfg.config['TANGERINE']['influx_org'], timeout=200000)
    try:
        query_api = client.query_api()
        tables = query_api.query('''
        from(bucket: "%s")
          |> range(start: %s, stop: %s)
          |> filter(fn: (r) => r["_measurement"] == "transactions")
          |> group()
          |> sort(columns: ["_time"], desc: false)
        ''' %(cfg.config['TANGERINE']['influx_bucket'], start_date, end_date))
        for table in tables:
            for row in table.records:
                if row['id'] not in old:                    
                    old[row['id']] = row  #could be deleted transaction, track it here
                elif row['_value'] != 0:
                    old[row['id']] = row  # we should only have one transaction with non-zero value
    finally:
        client.close()
    
    lst = []
    for row in old.values():
        if row['_value'] != 0: # only track non-deleted transactions
            lst.append((row['_time'], row['payee'],row['memo'], row['_value'], row['category'], row['id'], row['account']))
    df = pd.DataFrame(lst, columns = ['Date', 'Description', 'Memo', 'Amount', 'Category', 'Check#', 'Account'])
    if verbose:
        print(df)
    if dump:
        df.to_csv('transactions.csv', index=False)
        return    
                  
    client = InfluxDBClient(url=cfg.config['TANGERINE']['influx_url'], token=cfg.config['TANGERINE']['influx_token'], timeout=20000)            
    write_api = client.write_api(write_options=ASYNCHRONOUS)

    tt = transactions
    tt.sort(key=lambda x : x.date)
    for t in tt:
        date = t.date               
        if (date.date() < start_date or date.date() > end_date):
            if verbose:
                print('skipping (out of range)', t)
            continue
        
        skip = True if t.transaction_id in old else False  
        if verbose:
            print('skip', skip, t)

        point = Point("transactions")\
        .tag("payee", t.merchant_name) \
        .tag("memo", t.name) \
        .tag('account', t.account_id) \
        .tag('category', t.category[-1]) \
        .tag('id', t.transaction_id) \
        .field('amount', -1.0 * t.amount)  \
        .time(int(date.timestamp() * 10**9), WritePrecision.NS)
        if not skip:
            if verbose:
                print(point.to_line_protocol(), end='\n~~~~~~~~~~~~~~~~~~~~~~\n')
            if not dry:
                write_api.write(cfg.config['TANGERINE']['influx_bucket'], cfg.config['TANGERINE']['influx_org'], point)
    client.close()

In [59]:
start_date = df['Date'].min().date()
end_date = df['Date'].max().date()
dry = True
verbose = True
upload(transactions,cfg, start_date, end_date, verbose=verbose, dump=False, dry=dry)

                       Date                       Description  \
0 2024-09-13 04:00:00+00:00    Credit Card Rewards Redemption   
1 2024-09-13 04:00:00+00:00  EFT Withdrawal to 2600616RBC PYT   

                               Memo  Amount  Category  \
0    Credit Card Rewards Redemption  101.94    Credit   
1  EFT Withdrawal to 2600616RBC PYT -556.70  Mortgage   

                                  Check#     Account  
0  mQeBnKXXjZiO86PqkLQdC5vr3wQ6KaUm4Oxwa  depository  
1  MqYLOaDDP0syzYE7LKXJIqZYM7wAeOFBk13nx  depository  
skip False 2024-09-13 00:00:00 c0f172877f49863e9f4f88c0425d1c72 SBUX 84619 ZEHRS 538 GUELPH ON (SBUX 84619 ZEHRS 538 GUELPH ON) - -14.13  Restaurant
transactions,account=NA,category=\ Restaurant,id=c0f172877f49863e9f4f88c0425d1c72,memo=SBUX\ 84619\ ZEHRS\ 538\ GUELPH\ ON,payee=SBUX\ 84619\ ZEHRS\ 538\ GUELPH\ ON amount=14.13 1726185600000000000
~~~~~~~~~~~~~~~~~~~~~~
skip False 2024-09-13 00:00:00 4ea5822034ab5531ff8583b2512ef32c SP BOOST YOUR LUNCH GUELPH ON (SP